# MongoDB Inventory

## Container starten

Docker container starten mit dem Kernel PowerShell, danach auf den Kernel Python 3 switchen

In [1]:
docker start mongodb1

mongodb1


## Connection String

Connection String mit localhost und default Port abfüllen

In [21]:
connectionstring = "mongodb://0.0.0.0:27017"
import pymongo, dns, os, sys, csv, json
from datetime import datetime

## Database

Connect to MongoDB and list all database

In [2]:
# <-- run this function on a mac --> #
def get_dbs(connectionstring):
    '''Connect to MongoDB and list all database'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    print(mongo_client.list_database_names())
    mongo_client.close()

get_dbs(connectionstring)

['JupyterNB', 'admin', 'config', 'local', 'tinu']


## Documents

### Remove all documents

In [11]:
def delete_all_documents(connectionstring, database, collection, query, output = False):
    """Connect to MongoDB and delete all documents

    Args:
        connectionstring (String): Connection-string of the MongoDB
        database (String): Database name to export the data
        collection (String): Collection name to export the data
        query (JSON-String): Search for 
        output (bool, optional): Print output. Defaults to False.

    Example:
        credentials = input('user:password')
        connectionstring = "mongodb+srv://"+credentials+"@cluster0.epl3x.mongodb.net/?retryWrites=true&w=majority"
        delete_collection(connectionstring, 'JupyterNB', 'PoweredOffVMs', {'address':'India'}, True)
    """
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_col.delete_many(query)
    
    if output:
        for doc in mongo_col.find():
            print(doc)

    mongo_client.close()

delete_all_documents(connectionstring, 'tinu', 'inventory', {'DeviceType':''}, True)

### Import JSON

In [17]:
filetoimport = 'MssqlDB.tinu-inventory-2021523-13-00-00.json'

def import_json(json_file, connectionstring, database, collection):
    """Import data from JSON-file

    Args:
        json_file (String): FullName of the JSON-file to import
        connectionstring (String): Connection-string of the MongoDB
        database (String): Database name to import the data
        collection (String): Collection name to import the data
        
    Example:
        credentials = input('user:password')
        connectionstring = "mongodb+srv://"+credentials+"@cluster0.epl3x.mongodb.net/?retryWrites=true&w=majority"
        imp_json_file = "C:\\Users\\Tinu\\Downloads\\poweredOffVMs.json"
        import_json(imp_json_file, connectionstring, 'JupyterNB', 'PoweredOffVMs')
    """
    check_path = os.path.exists(json_file)
    if(check_path == True):
        import pymongo
        mongo_client = pymongo.MongoClient(connectionstring)
        mongo_db     = mongo_client[database]
        mongo_col    = mongo_db[collection]

        with open(json_file, "r") as read_file:
            json_data = json.load(read_file)

        # It is important to remove the _id field in order to import it into mongodb
        for element in json_data:
            element.pop('_id', None)

        try:
            if isinstance(json_data, list):
                mongo_col.insert_many(json_data)  
            else:
                mongo_col.insert_one(json_data)
            
        except:
            sys.exit('file {}'.format(json_file))
                
        mongo_client.close()

    else:
        print(f'[WARN] File not found: {json_file}')

# Join the path to Downloads
if sys.platform == "linux" or sys.platform == "linux2":
    home_dir = os.environ['HOME']
elif sys.platform == "darwin":
    home_dir = os.environ['HOME']
elif sys.platform == "win32":
    home_dir  = os.environ['USERPROFILE']
downloads = os.path.join(home_dir,'Downloads')

json_file = os.path.join(downloads, filetoimport)
import_json(json_file, connectionstring, 'tinu', 'inventory')

### List all documents

In [28]:
# <-- Print out all documents --> #
def get_documents(connectionstring, database, collection):
    '''Connect to MongoDB and return data'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_client.close()
    return mongo_col.find()

import pandas as pd
table = []

result = get_documents(connectionstring, 'tinu', 'inventory')
for doc in result:
    #print(doc)
    thisdict = { 
        'ID'             : doc['ID'],
        'RunTime'        : doc['RunTime'],
        'DeviceType'     : doc['DeviceType'],
        'DeviceName'     : doc['DeviceName'],
        'Physical cores' : doc['Physical cores'],
        'MemoryGB'       : doc['MemoryGB'],
        'DiskspaceGB'    : doc['DiskspaceGB'],
        'PurchaseDate'   : doc['PurchaseDate'],
        'Price'          : doc['Price'],
        'Payment'        : doc['Payment'],
        'Warranty'       : doc['Warranty'],
        'Link'           : doc['Link'],
    }
    table.append(thisdict)

df = pd.DataFrame(table)
df.head()

,ID,RunTime,DeviceType,DeviceName,Physical cores,MemoryGB,DiskspaceGB,PurchaseDate,Price,Payment,Warranty,Link
0,96fb423e-f36b-1410-8134-00daf9a34b4e,2021-05-21T15:00:24.09,Convertible,Microsoft Surface Pro 4,2.0,8.0,256.0,2015-05-13T00:00:00,NaN,Maestrocard,2,None
1,97fb423e-f36b-1410-8134-00daf9a34b4e,2021-05-22T17:04:13.177,iMac,"Apple iMac (Retina 4K, 21.5-inch, 2017)",4.0,16.0,512.0,2019-07-01T00:00:00,2341.4,Mastercard,2+1,None
2,4a27433e-f36b-1410-8136-00daf9a34b4e,2021-05-22T17:38:48.047,Handy,Huawei P30 Pro DualSIM 4G+ Aurora,8.0,8.0,128.0,2019-05-13T00:00:00,999.0,Mastercard,2+1,None
3,4e27433e-f36b-1410-8136-00daf9a34b4e,2021-05-21T16:30:57.48,Camera,NIKON Z6,NaN,NaN,NaN,2020-01-15T00:00:00,2099.0,Mastercard,3+1,None
4,82c7423e-f36b-1410-8138-00daf9a34b4e,2021-05-22T17:23:32.503,Convertible,HP EliteBook x360 1030 G8,4.0,16.0,512.0,2021-05-29T80:00:00,2249.0,Mastercard,3+1,https://store.hp.com/SwitzerlandStore/Merch/Pr...


### Update document

### Insert one document

In [29]:
documenttoimport = { 
    "RunTime": datetime.strptime(doc['Datum'], '%d.%m.%Y', 
    "DeviceType": Notebook,
    "DeviceName": "ThinkPad"
    "Physical cores": 4,
    "MemoryGB": 16,
    "DiskspaceGB": 512,
    "PurchaseDate": datetime.strptime(doc['Datum'], '%d.%m.%Y',
    "Payment": "Mastercard",
    "Warranty": "3+1",
    "Link": "https://tinuwalther.github.io"
}

def add_document(connectionstring, mongodatabase, collection, document):
    '''Connect to MongoDB and add a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    mongo_col.insert_one(document)
    for doc in mongo_col.find():
        print(doc)

    mongo_client.close()

add_document(connectionstring, 'tinu', 'inventory', documenttoimport)

SyntaxError: invalid syntax (<ipython-input-29-6f3f82e43887>, line 3)

## Container stoppen

Kernel PowerShell switchen und docker container stoppen

In [1]:
docker stop mongodb1

mongodb1
